In [1]:
# Load packages
from dotenv import load_dotenv

load_dotenv()
# Dataset
from datasets import load_dataset
# For EDA
from collections import Counter
import matplotlib.pyplot as plt
# For Model training
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer , AdamW , get_linear_schedule_with_warmup
# For Evaluation
import numpy as np
import arrow
import math
import evaluate
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from transformers import RobertaForSequenceClassification

/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Training info
dataset_id = "PolyAI/banking77"  # Training dataset
model_id = "bert-large-uncased"  # For tokenize and modeling

# Load everythings for training
dataset = load_dataset(dataset_id)

# Check dataset
print(dataset) # train: 10003 rows | test: 3080 rows

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3080
    })
})


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# tokenize函數 用來批次將資料轉換成input tokens
def tokenize_function(data):
    return tokenizer(
        data["text"],
        # add_special_tokens=True, # Default = True [CLS], [SEP]
        padding="max_length", # 補值到模型可接受的最大值
        max_length=64, # 設定裁切長度
        return_attention_mask=True,
        truncation=True # 是否裁切(要設定max_length才會裁切)
    )

# Tokenize
tokenized_datasets = dataset.map(tokenize_function, batched=True) # 將dataset轉換為input tokens

Map: 100%|██████████| 3080/3080 [00:00<00:00, 31566.48 examples/s]


In [7]:
# Train-Valid-Test split
train_test_split = tokenized_datasets["train"].shuffle(seed=66).train_test_split(test_size=0.1, seed=66)
train_dataset, valid_dataset = train_test_split["train"], train_test_split["test"]
# Test dataset remains the same
test_dataset = tokenized_datasets["test"].shuffle(seed=66)

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=77)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
    

In [10]:
lr = [5e-6,1e-5,2e-5,5e-5,1e-4]

In [11]:
def test6_training(model,lr=2e-5,epoch_num=10):
    train_batch_size = 16
    eval_batch_size=64
    
    training_args = TrainingArguments(
        output_dir="result/{}_{}({})".format("bert_large",arrow.now().format('MM-DD_HH:mm'),lr),
        evaluation_strategy="epoch",         # 每個 epoch 評估一次
        save_strategy="epoch",               # 每個 epoch 保存一次模型
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=eval_batch_size,
        num_train_epochs=epoch_num,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=200,
        save_total_limit=1,                  # 最多保留3個模型checkpoint
        load_best_model_at_end=True,         # 在訓練結束時加載最佳模型
        metric_for_best_model="accuracy",    # 根據準確率選擇最佳模型
        learning_rate=lr,
        label_smoothing_factor=0.1,
        # fp16=True,               # Only use with GPU
        # label_names=["labels"]         #不知道為什麼加了這個就可以
        )
    
    trainer = Trainer(
        model=model ,
        args=training_args,
        train_dataset=train_dataset,  # 替換為你的訓練數據集
        eval_dataset=valid_dataset,    # 替換為你的驗證數據集
        compute_metrics=compute_metrics
        )

    trainer.train()
    
    return trainer.evaluate(test_dataset)


result_6 = {}
for l in lr:
    print("-------------------------{}-------------------------".format(l))
    model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels=77)
    result_6[l] = test6_training(model,lr=l)

-------------------------5e-06-------------------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to av

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,4.268000,3.875613,0.164835,0.126045,0.207236,0.164835
2,3.021200,2.318401,0.748252,0.733227,0.801143,0.748252
3,1.919200,1.550483,0.872128,0.868771,0.886341,0.872128
4,1.399200,1.260938,0.892108,0.890082,0.900142,0.892108
5,1.161800,1.142111,0.902098,0.901294,0.912744,0.902098
6,1.073600,1.080347,0.915085,0.914562,0.920614,0.915085
7,0.974800,1.049585,0.913087,0.912551,0.918602,0.913087
8,0.936700,1.042152,0.916084,0.915506,0.921912,0.916084
9,0.919600,1.039192,0.919081,0.918480,0.925169,0.919081
10,0.898400,1.036618,0.917083,0.916412,0.923132,0.917083


/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

-------------------------1e-05-------------------------


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,2.841000,1.283785,0.850150,0.850547,0.881081,0.850150
2,1.095700,1.077867,0.898102,0.898482,0.907328,0.898102
3,0.958500,1.046949,0.915085,0.915139,0.921758,0.915085
4,0.890200,1.042714,0.912088,0.912316,0.919769,0.912088
5,0.824700,1.058806,0.918082,0.918626,0.930233,0.918082
6,0.795600,1.075254,0.913087,0.912867,0.918187,0.913087
7,0.773500,1.055456,0.926074,0.926146,0.932572,0.926074
8,0.762800,1.062663,0.917083,0.916883,0.922395,0.917083
9,0.759000,1.057364,0.929071,0.928980,0.934128,0.929071
10,0.756200,1.059712,0.924076,0.924024,0.929181,0.924076


-------------------------0.0001-------------------------


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,2.159800,1.363604,0.802198,0.797696,0.837413,0.802198
2,1.189700,1.179218,0.872128,0.870128,0.891297,0.872128
3,1.100000,1.129114,0.894106,0.894881,0.908407,0.894106
4,1.074500,1.126566,0.884116,0.879997,0.905364,0.884116
5,0.968200,1.124807,0.890110,0.888171,0.908048,0.890110
6,0.909500,1.080735,0.896104,0.893955,0.910454,0.896104
7,0.873400,1.037842,0.919081,0.917024,0.931099,0.919081
8,0.832200,1.075225,0.908092,0.904864,0.909526,0.908092
9,0.804300,1.069684,0.914086,0.911339,0.915416,0.914086
10,0.790700,1.071712,0.916084,0.913492,0.917088,0.916084


/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dykos0508/.virtualenvs/torch_s/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize(

In [12]:
result_6

{5e-06: {'eval_loss': 1.031304955482483,
  'eval_accuracy': 0.9191558441558442,
  'eval_f1': 0.9193469558321538,
  'eval_precision': 0.9236795585187275,
  'eval_recall': 0.9191558441558442,
  'eval_runtime': 7.9967,
  'eval_samples_per_second': 385.157,
  'eval_steps_per_second': 6.127,
  'epoch': 10.0},
 1e-05: {'eval_loss': 1.005737066268921,
  'eval_accuracy': 0.9321428571428572,
  'eval_f1': 0.9321020112322986,
  'eval_precision': 0.9345598987730979,
  'eval_recall': 0.9321428571428572,
  'eval_runtime': 8.0073,
  'eval_samples_per_second': 384.649,
  'eval_steps_per_second': 6.119,
  'epoch': 10.0},
 2e-05: {'eval_loss': 1.0119210481643677,
  'eval_accuracy': 0.9344155844155844,
  'eval_f1': 0.9346092172364635,
  'eval_precision': 0.9375294340581146,
  'eval_recall': 0.9344155844155844,
  'eval_runtime': 7.9758,
  'eval_samples_per_second': 386.169,
  'eval_steps_per_second': 6.144,
  'epoch': 10.0},
 5e-05: {'eval_loss': 1.0390645265579224,
  'eval_accuracy': 0.9308441558441558,
